In [ ]:
'''
Setup
1. Import Library
2. Import File
3. Cleaning data
4. Feature Engineering
5. Buat Model
6. Tes Model
'''

In [2]:
#Import Library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder

from mlxtend.preprocessing import minmax_scaling

print("Setup done")

Setup done


In [ ]:
#Import File
data= pd.read_csv("C:/Users/visco/Downloads/Housing Prize/train.csv")
print(data.shape)
data.iloc[:,20:30]

In [ ]:
#Liat data hilang
data_hilang = data.isnull().sum()
data_hilang[70:81]

In [ ]:
#Hapus data Hilang
data_bersih = data.dropna(axis= 1) #Hapus kolom yang punya missing values
print(f'Columns Before Data Cleaning: {data.shape[1]}')
print(f'Columns After Data Cleaning: {data_bersih.shape[1]}')
print(data_bersih.shape)

In [ ]:
#Kolom yang dihapus
colbf= data.columns
colaf=data_bersih.columns

dropped_cols = set(colbf)-set(colaf)
print(dropped_cols)

In [ ]:
#Cek Kolom
print(data['Functional'])

In [ ]:
#One-Hot Decoding
data_bersih['BldgType'] = data_bersih['BldgType'].astype('category')
data_bersih = pd.get_dummies(data_bersih ,columns= ['BldgType'], prefix='BldgType')
data_bersih['HouseStyle'] = data_bersih['HouseStyle'].astype('category')
data_bersih = pd.get_dummies(data_bersih ,columns= ['HouseStyle'], prefix='HouseStyle')

In [ ]:
#Mapping
map_exter = {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1}
data_bersih['ExterQual'] = data_bersih['ExterQual'].map(map_exter)
data_bersih['ExterCond'] = data_bersih['ExterCond'].map(map_exter)
data_bersih['KitchenQual'] = data_bersih['KitchenQual'].map(map_exter) #Sama
data_bersih['HeatingQC'] = data_bersih['HeatingQC'].map(map_exter) #Sama


In [ ]:
#cek kolom after one-hot
print(f'Dimensi Data setelah One-Hot: {data_bersih.shape}')
[col for col in data_bersih.columns if 'HouseStyle' in col]

a=0
for c in data_bersih['HouseStyle_2.5Fin']:
    if c==True:
        a+=1
print(a)

In [ ]:
#Pemilihan Kolom untuk train_data
y = data_bersih.SalePrice
col_train = ['MSSubClass',
             'BldgType_1Fam',
             'BldgType_2fmCon',
             'BldgType_Duplex',
             'BldgType_Twnhs',
             'BldgType_TwnhsE', 
             'HouseStyle_1.5Fin',
             'HouseStyle_1.5Unf',
             'HouseStyle_1Story',
             'HouseStyle_2.5Fin',
             'HouseStyle_2.5Unf',
             'HouseStyle_2Story',
             'HouseStyle_SFoyer',
             'HouseStyle_SLvl', 
             'OverallQual', 
             'OverallCond', 
             'YearBuilt',
             'YearRemodAdd', 
             'HeatingQC',
             'GrLivArea',
             'FullBath',
             'HalfBath',
             'BedroomAbvGr',
             'KitchenAbvGr',
             'GarageCars',
             'WoodDeckSF',
             'OpenPorchSF',
             'ScreenPorch',
             'EnclosedPorch',
             'TotRmsAbvGrd',
             'MiscVal',
             'YrSold']
X = data_bersih[col_train]

In [ ]:
#train
train_X, val_X, train_y, val_y = train_test_split(X, y, test_size=0.2,random_state=1)
model = RandomForestRegressor(n_estimators= 300,random_state = 1)

model.fit(train_X,train_y)
vald_predict = model.predict(val_X)
#Model Validation
mea =mean_absolute_error(vald_predict, val_y)
print("Validation MAE for Random Forest Model: {:,.0f}".format(mea))

In [ ]:
#Cek fitur penting
pi=pd.Series(model.feature_importances_, index=train_X.columns).sort_values(ascending=False)
print(pi)

In [ ]:
#Finishing
#Full data
model_test = RandomForestRegressor(max_depth=20, random_state=1)
model_test.fit(X,y)

In [ ]:
test_data = pd.read_csv('C:/Users/visco/Downloads/Housing Prize/test.csv')

#Nambahin kolom onehot
test_data['BldgType'] = test_data['BldgType'].astype('category')
test_data = pd.get_dummies(test_data ,columns= ['BldgType'], prefix='BldgType')
test_data['HouseStyle'] = test_data['HouseStyle'].astype('category')
test_data = pd.get_dummies(test_data ,columns= ['HouseStyle'], prefix='HouseStyle')

#Mapping
map_exter = {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1}
test_data['ExterQual'] = test_data['ExterQual'].map(map_exter)
test_data['ExterCond'] = test_data['ExterCond'].map(map_exter)
test_data['KitchenQual'] = test_data['KitchenQual'].map(map_exter) #Sama
test_data['HeatingQC'] = test_data['HeatingQC'].map(map_exter) #Sama

In [ ]:
[col for col in test_data.columns if 'HouseStyle' in col]

In [ ]:
test_X = test_data[col_train]
test_pred = model_test.predict(test_X)

In [ ]:
output = pd.DataFrame({'Id' : test_data.Id, 'SalePrice' : test_pred})

In [ ]:
output.to_csv('Housing Prize entah entah.csv',index=False)